In [3]:

('mem[8] = 11'.split(' ')[0]).replace('[', '').replace(']', '').replace('mem', '')

'8'

In [36]:
inFile = open("input", 'r') 
lines = inFile.readlines()
lines = [l.replace('\n', '') for l in lines]

# sampleLines = [
#     "mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X",
#     "mem[8] = 11", 
#     "mem[7] = 101", 
#     "mem[8] = 0"
# ]

sampleLines = [
    "mask = 000000000000000000000000000000X1001X",
    "mem[42] = 100",
    "mask = 00000000000000000000000000000000X0XX",
    "mem[26] = 1"
]

# def constructMaskGroups(lines): 
#     nodes = {}
#     memList = []
#     for line in lines: 
#         if 'mask' in line: 
#             maskString = ((line.split(' '))[2])
#             maskNode = {}
#             for i in range(len(maskString)): 
#                 if maskString[i] != 'X': 
#                     maskNode[i] = maskString[i]
#             nodes['mask'] = maskNode 
#         else: 
#             memLoc = (line.split(' ')[0]).remove('')


# | followed by & the bits 
def constructMaskGroups(lines): 
    nodeList = []
    # currNode = 0
    for line in lines: 
        if "mask" in line: 
            nodeList.append({})
            maskNode = {}
            maskNode['maskDecOR'] = int(((line.split(' '))[2]).replace('X', '0'), 2)
            maskNode['maskDecAND'] = int(((line.split(' '))[2]).replace('X', '1'), 2)
            maskNode['maskStr'] = (line.split(' '))[2]
            nodeList[len(nodeList) - 1]['mask'] = maskNode
            nodeList[len(nodeList) - 1]['memList'] = []
        else:
            memNode = {}
            memNode['loc'] = int((line.split(' ')[0]).replace('[', '').replace(']', '').replace('mem', ''))
            memNode['val'] = int(line.split(' ')[2])
            nodeList[len(nodeList) - 1]['memList'].append(memNode)
    return nodeList 

nodeList = constructMaskGroups(lines)
sampleNodes = constructMaskGroups(sampleLines)
print(sampleNodes)

[{'mask': {'maskDecOR': 18, 'maskDecAND': 51, 'maskStr': '000000000000000000000000000000X1001X'}, 'memList': [{'loc': 42, 'val': 100}]}, {'mask': {'maskDecOR': 0, 'maskDecAND': 11, 'maskStr': '00000000000000000000000000000000X0XX'}, 'memList': [{'loc': 26, 'val': 1}]}]


In [48]:
from itertools import product

def generateLocs(mask, address): 
    locs = []
    newAddress = [] 
    Xindices = []
    addressStr = list(bin(address))[2:]
    address = (['0']*(len(mask)-len(addressStr)))+[a for a in addressStr]
    # print(address)
    for i in range(len(mask)): 
        m = mask[i]
        a = address[i]
        if m == '0':
            newAddress.append(a)
        if m == '1': 
            newAddress.append(m)
        if m == 'X': 
            newAddress.append('X')
            Xindices.append(i)
    # print(newAddress)
    permutations = list(product('01', repeat=len(Xindices)))
    for p in permutations: 
        newLoc = newAddress.copy()
        for i in range(len(p)):
            newLoc[Xindices[i]] = p[i]
        locs.append(int('0b' + ''.join(newLoc) ,2))
    # print(locs)
    return locs 

def modAddresses(nodeList):
    addresses = {}
    for node in nodeList: 
        for mem in node['memList']:
            loc = mem['loc']
            # storedVal = (mem['val'] | node['mask']['maskDecOR']) & node['mask']['maskDecAND']
            locs = generateLocs(node['mask']['maskStr'], mem['loc'])
            for l in locs: 
                addresses[l] = mem['val']
    return addresses

def getSum(addresses): 
    total = 0
    for address in addresses: 
        total += addresses[address]
    return total
    

In [49]:
mods = modAddresses(nodeList)

getSum(mods)

4288986482164